In [5]:
from keras.models import Sequential
from keras.layers import Convolution2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import BatchNormalization
from keras.datasets import cifar10
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import OneHotEncoder
from keras import regularizers


In [6]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

print(x_train.shape)  # (50000, 32, 32, 3)

## Normalize Data
def normalize(X_train, X_test):
    mean = np.mean(X_train, axis=(0, 1, 2, 3))
    std = np.std(X_train, axis=(0, 1, 2, 3))
    X_train = (X_train - mean) / (std + 1e-7)
    X_test = (X_test - mean) / (std + 1e-7)
    return X_train, X_test, mean, std

## Normalize Training and Testset
x_train, x_test, mean_train, std_train = normalize(x_train, x_test)

(50000, 32, 32, 3)


In [7]:
## OneHot Label 由(None, 1)-(None, 10)
## ex. label=2,變成[0,0,1,0,0,0,0,0,0,0]
one_hot=OneHotEncoder()
y_train=one_hot.fit_transform(y_train).toarray()
y_test=one_hot.transform(y_test).toarray()

C:\Anaconda3\lib\site-packages\sklearn\preprocessing\_encoders.py:415: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)


In [8]:
classifier=Sequential()

#卷積組合
classifier.add(Convolution2D(32,3,3,input_shape=(32,32,3),activation='relu'))  #32,3,3,input_shape=(32,32,3),activation='relu''
classifier.add(BatchNormalization())

'''自己決定MaxPooling2D放在哪裡'''
classifier.add(MaxPooling2D(pool_size=(2,2)))

#卷積組合
classifier.add(Convolution2D(32,3,3,input_shape=(32,32,3),activation='relu'))
classifier.add(BatchNormalization())
classifier.add(MaxPooling2D(pool_size=(2,2)))

#flatten
classifier.add(Flatten())

#FC
classifier.add(Dense(output_dim=100,activation='relu',kernel_regularizer=regularizers.l2(l=0.001))) ##regularizers
classifier.add(BatchNormalization()) ##BatchNormalization
classifier.add(Dropout(p=0.5)) ##Dropout

classifier.add(Dense(output_dim=100,activation='relu',kernel_regularizer=regularizers.l2(0.001)))##regularizers
classifier.add(BatchNormalization()) ##BatchNormalization

classifier.add(Dropout(p=0.3))##Dropout

classifier.add(Dense(output_dim=10,activation='softmax'))


#輸出
classifier.add(Dense(output_dim=10,activation='relu'))



#超過兩個就要選categorical_crossentrophy
classifier.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])


from keras.preprocessing.image import ImageDataGenerator ##Augmentation
img_gen = ImageDataGenerator( featurewise_center=True,featurewise_std_normalization=True,rotation_range=10,width_shift_range=0.1,
                                            height_shift_range=0.1,shear_range=0.1,zoom_range=0.1,horizontal_flip=True,vertical_flip=False)
img_gen.fit(x_train)


from keras.callbacks import EarlyStopping
earlystop = EarlyStopping(monitor='test_loss', patience=8, verbose=1) ##earlystop

##開始訓練
classifier.fit_generator(img_gen.flow(x_train, y_train, batch_size=100) ,steps_per_epoch=500,
                               epochs=100, validation_data = (x_test, y_test),callbacks = [earlystop])

C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (3, 3), input_shape=(32, 32, 3..., activation="relu")`
  after removing the cwd from sys.path.
C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:11: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (3, 3), input_shape=(32, 32, 3..., activation="relu")`
  # This is added back by InteractiveShellApp.init_path()
C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:19: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="relu", kernel_regularizer=<keras.reg..., units=100)`
C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:21: UserWarning: Update your `Dropout` call to the Keras 2 API: `Dropout(rate=0.5)`
C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:23: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="relu", kernel_regularizer=<keras.reg..., units=100)`
C:\Anaconda3\lib\site-pack

Epoch 1/100
500/500 [==============================] - 337s 675ms/step - loss: nan - acc: 0.1002 - val_loss: nan - val_acc: 0.1000
Epoch 2/100


C:\Anaconda3\lib\site-packages\keras\callbacks.py:535: RuntimeWarning: Early stopping conditioned on metric `test_loss` which is not available. Available metrics are: val_loss,val_acc,loss,acc
  (self.monitor, ','.join(list(logs.keys()))), RuntimeWarning


 89/500 [====>.........................] - ETA: 4:35 - loss: nan - acc: 0.1006

KeyboardInterrupt: 

In [ ]:
input_example = (np.zeros(shape=(1, 32, 32, 3)) - mean_train) / (std_train + 1e-7)
classifier.predict(input_example)